In [1]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from pathlib import Path
import numpy as np
import PIL
import PIL.Image
import json
import pandas as pd

#change working directory to root
ROOT_DIR = os.getcwd()
while os.path.basename(ROOT_DIR) != 'VisIrNet':
    ROOT_DIR = os.path.abspath(os.path.join(ROOT_DIR,'..'))
sys.path.insert(0,ROOT_DIR)
os.chdir(ROOT_DIR)

ROOT_DIR = Path(ROOT_DIR)

print(tf.__version__)
devices = tf.config.list_physical_devices('GPU')
print("len(devices): ", len(devices))
print(f"available GPUs: {devices}");

from collections import defaultdict

2.14.0
len(devices):  1
available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# extract important metrics from logs

logs_path = ROOT_DIR / 'logs'



## backbone 

```python

## helper functions

def parse_backbone_log(logs):
    """
        'model-name',
        'epochs',
        'loss_function',
        'resumed_from',
        'train_size',
        'test_size',
        'tag_name',
        'per_epoch_metrics',
        'training_time'
    """
    #
    interested_in = ['fir_frgb', 'frgb_Irgb', 'fir_Irgb']

    values_dict = defaultdict()
    values_dict['loss'] = logs["loss_function"]
    # exatract the initial and final values
    logs = logs["per_epoch_metrics"]["test_results"]

    inital_values= { k: v[0] for k,v in logs.items() if k in interested_in}
    final_values= { k: v[-1] for k,v in logs.items() if k in interested_in}

    
    values_dict.update({f"i-{k}": v for k,v in inital_values.items()})
    values_dict.update({f"f-{k}": v for k,v in final_values.items()})
    return dict(values_dict)


def handle_backbone(log_dir, backbone_logs):
    
    backbone_parsed_dict = defaultdict(list)
    
    for b_logs in backbone_logs:        
        # read the json 
        with open(log_dir / b_logs) as f:
            example_logs = json.load(f)
            parsed_dict = parse_backbone_log(example_logs)
        
        # add the parsed dict to the backbone_parsed_dict
        for k,v in parsed_dict.items():
            backbone_parsed_dict[k].append(v)
            
    return dict(backbone_parsed_dict)
    
    
# get all directories in logs_path
log_dirs = [x for x in logs_path.iterdir() if x.is_dir() and x.name != 'tensorboard']
dataset_with_logs = [d.name for d in log_dirs]



for log_dir, d_name in zip(log_dirs,dataset_with_logs):
    # per dataset logs
    per_dataset_logs = [x.name for x in log_dir.glob('*.json')]

    backbone_logs = list(filter(lambda x: 'regressionBlock' not in x,per_dataset_logs))
    regressionBlock_logs = list(filter(lambda x: 'regressionBlock' in x,per_dataset_logs))


    backbone_parsed = handle_backbone(log_dir, backbone_logs)
    # regression_parsed = handle_regression(log_dir, regressionBlock_logs)
    
    
    print("dataset_name: ", d_name)

    backbone_logs_df = pd.DataFrame(backbone_parsed)
    display(backbone_logs_df)

```

## Regression Head

In [3]:
## helper functions

def parse_regression_log(logs):
    """
    'model-name',
    'epochs',
    'resumed_from',
    'train_size',
    'test_size',
    'featureEmbeddingBackBone',
    'predicting_homography',
    'tag_name',
    'per_epoch_metrics',
                        'l1_homography_loss',
                        'l2_homography_loss',
                        'l1_corners_loss',
                        'l2_corners_loss',
                        'total_loss'
    'training_time',
    'featureEmbeddingBackbone'
    """
    #
    # interested_in = ['l1_homography_loss',
    #                     'l2_homography_loss',
    #                     'l1_corners_loss',
    #                     'l2_corners_loss',]
    interested_in = ['l2_corners_loss',]
    tag = logs["tag_name"]
    backbone_loss = tag.split('-') [2]

    
    values_dict = defaultdict()
    values_dict['backbone'] = backbone_loss
    # exatract the initial and final values
    logs = logs["per_epoch_metrics"]["test_results"]

    inital_values= { k: v[0] for k,v in logs.items() if k in interested_in}
    final_values= { k: v[-1] for k,v in logs.items() if k in interested_in}
    
    epochs = len(list(logs.values())[0])
    values_dict.update({f"epochs": epochs})
    values_dict.update({f"i-{k}": v for k,v in inital_values.items()})
    values_dict.update({f"{k}": np.sqrt(float(v)) for k,v in final_values.items()})
    return dict(values_dict)


def handle_regression_head(log_dir, logs):
    
    _parsed_dict = defaultdict(list)
    

    
    for b_logs in logs:        
        # read the json 
        with open(log_dir / b_logs) as f:
            example_logs = json.load(f)
            parsed_dict = parse_regression_log(example_logs)
            
        
        # add the parsed dict to the backbone_parsed_dict
        for k,v in parsed_dict.items():
            _parsed_dict[k].append(v)
            
    return dict(_parsed_dict)
    

**l2_corners_loss**

In [4]:
# get all directories in logs_path
print("l2_corners_loss")
log_dirs = [x for x in logs_path.iterdir() if x.is_dir() and x.name != 'tensorboard']
dataset_with_logs = [d.name for d in log_dirs]

for log_dir, d_name in zip(log_dirs,dataset_with_logs):
    # per dataset logs
    per_dataset_logs = [x.name for x in log_dir.glob('*.json')]

    # backbone_logs = list(filter(lambda x: 'regressionBlock' not in x,per_dataset_logs))
    regressionBlock_logs = list(filter(lambda x: 'regressionBlock' in x and "l2_corners_loss" in x ,per_dataset_logs))


    _parsed = handle_regression_head(log_dir, regressionBlock_logs)
    # regression_parsed = handle_regression(log_dir, regressionBlock_logs)
    
    
    print("dataset_name: ", d_name)
    
    # save_as = Path("resources/results") 
    # save_as.mkdir(parents=True, exist_ok=True)
    _logs_df = pd.DataFrame(_parsed)
    display(_logs_df)
    
    # display(list(regressionBlock_logs))



l2_corners_loss
dataset_name:  VEDAI


,backbone,epochs,i-l2_corners_loss,l2_corners_loss
0,mae_pixel,30,895.644409,18.533229
1,ssim_pixel,30,632.119995,19.114550
2,mse_pixel,30,499.333313,19.140575
3,sse_pixel,30,1217.907715,20.671191


dataset_name:  SkyData


,backbone,epochs,i-l2_corners_loss,l2_corners_loss
0,ssim_pixel,30,509.250122,18.505228
1,mae_pixel,30,357.603638,18.499010
2,mse_pixel,30,370.594299,18.590597
3,sse_pixel,30,938.347839,19.071471


**l2_homography_loss**

In [5]:
# get all directories in logs_path
print("l2_homography_loss")
log_dirs = [x for x in logs_path.iterdir() if x.is_dir() and x.name != 'tensorboard']
dataset_with_logs = [d.name for d in log_dirs]

for log_dir, d_name in zip(log_dirs,dataset_with_logs):
    # per dataset logs
    per_dataset_logs = [x.name for x in log_dir.glob('*.json')]

    # backbone_logs = list(filter(lambda x: 'regressionBlock' not in x,per_dataset_logs))
    regressionBlock_logs = list(filter(lambda x: 'regressionBlock' in x and "l2_corners_loss" not in x ,per_dataset_logs))


    _parsed = handle_regression_head(log_dir, regressionBlock_logs)
    # regression_parsed = handle_regression(log_dir, regressionBlock_logs)
    
    
    print("dataset_name: ", d_name)

    _logs_df = pd.DataFrame(_parsed)
    display(_logs_df)
    
    # display(list(regressionBlock_logs))


l2_homography_loss
dataset_name:  VEDAI


,backbone,epochs,i-l2_corners_loss,l2_corners_loss
0,mse_pixel,30,20236.347656,128.310179
1,sse_pixel,30,38462.300781,459.266212
2,mae_pixel,30,1281.466919,178.071395
3,ssim_pixel,30,19303.982422,134.179705


dataset_name:  SkyData


,backbone,epochs,i-l2_corners_loss,l2_corners_loss
0,mae_pixel,30,13033.645508,35.835938
1,ssim_pixel,30,5879.158203,20.105120
2,sse_pixel,30,14832.153320,48.961109
3,mse_pixel,30,13933.938477,21.559428
